In [41]:
import sqlite3
import subprocess
import json
from datetime import datetime
import os

In [42]:
prompt = "Hi, how are you doing today? Please tell me a joke."

result = subprocess.run(
    ["ollama", "run", "llama3.2:3b"],
    input=prompt.encode("utf-8"),
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

print(result.stdout.decode("utf-8", errors="ignore").strip())


I'm doing well, thank you for asking! I'd be happy to share a joke with you. Here's one:

What do you call a fake noodle?

(wait for it...)

An impasta!

Hope that made you smile! Do you want to hear another one?


In [43]:
DB_PATH = "timesheet.db"

if os.path.exists(DB_PATH):
    os.remove(DB_PATH)

print("Database reset")


Database reset


In [44]:
def get_today_table():
    return "timesheet_" + datetime.now().strftime("%Y_%m_%d")

def ensure_table():
    table = get_today_table()
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()

    cur.execute(f"""
    CREATE TABLE IF NOT EXISTS {table} (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        task TEXT,
        duration_minutes INTEGER
    )
    """)

    conn.commit()
    conn.close()

ensure_table()
print("Table ready")


Table ready


In [45]:
def show_timesheet():
    table = get_today_table()
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()

    cur.execute(f"SELECT task, duration_minutes FROM {table}")
    rows = cur.fetchall()

    print("\nTimesheet:\n")

    if not rows:
        print("(empty)")
    else:
        for t, d in rows:
            print(f"{t} — {d} min")

    conn.close()

show_timesheet()



Timesheet:

(empty)


In [46]:
def infer_task(text):
    prompt = f"""
Summarize user's activity into ONE HIGH-LEVEL TASK for timesheet.

Rules:
- Return ONLY JSON
- 2–4 words
- Ignore small differences
- Similar meanings must map to SAME task

IMPORTANT NORMALIZATION:
- "reviewing docs", "reading documentation", "checking docs" → "reading documentation"
- Coding/debugging/automation → "automating timesheet system"

Format:
{{ "task": "task name" }}

Text:
{text[:1500]}
"""

    result = subprocess.run(
        ["ollama", "run", "llama3.2:3b"],
        input=prompt.encode("utf-8"),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )

    out = result.stdout.decode("utf-8", errors="ignore").strip()

    try:
        task = json.loads(out)["task"].lower().strip()
    except:
        task = "general work"

    # --- Manual normalization (extra safety) ---
    if "doc" in task:
        task = "reading documentation"
    if "debug" in task or "script" in task:
        task = "automating timesheet system"
    if "automation" in task or "coding" in task:
        task = "automating timesheet system"

    return task


In [47]:
last_time = None
last_task = None

def update_db(task, current_time):
    global last_time, last_task

    table = get_today_table()
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()

    if last_time is None:
    # Assume first interval = 5 min
        table = get_today_table()
        cur.execute(f"INSERT INTO {table} (task, duration_minutes) VALUES (?, ?)", (task, 5))
        conn.commit()
        conn.close()
    
        last_time = current_time
        last_task = task
        return


    delta = int((current_time - last_time).total_seconds() / 60)
    delta = max(delta, 1)

    if task == last_task:
        cur.execute(f"SELECT id, duration_minutes FROM {table} ORDER BY id DESC LIMIT 1")
        row = cur.fetchone()

        if row:
            new_d = row[1] + delta
            cur.execute(f"UPDATE {table} SET duration_minutes=? WHERE id=?", (new_d, row[0]))
        else:
            cur.execute(f"INSERT INTO {table} (task, duration_minutes) VALUES (?,?)", (task, delta))
    else:
        cur.execute(f"INSERT INTO {table} (task, duration_minutes) VALUES (?,?)", (task, delta))

    conn.commit()
    conn.close()

    last_time = current_time
    last_task = task


In [48]:
timeline = [
    ("2026-02-16 10:00:00", "writing python automation code"),
    ("2026-02-16 10:05:00", "debugging script error"),
    ("2026-02-16 10:10:00", "reading documentation"),
    ("2026-02-16 10:15:00", "reading documentation"),
    ("2026-02-16 10:20:00", "working on automation code"),
]

for ts, text in timeline:
    t = datetime.strptime(ts, "%Y-%m-%d %H:%M:%S")
    task = infer_task(text)

    print(f"{ts} → {task}")
    update_db(task, t)

show_timesheet()


2026-02-16 10:00:00 → automating timesheet system
2026-02-16 10:05:00 → automating timesheet system
2026-02-16 10:10:00 → reading documentation
2026-02-16 10:15:00 → reading documentation
2026-02-16 10:20:00 → automating timesheet system

Timesheet:

automating timesheet system — 10 min
reading documentation — 10 min
automating timesheet system — 5 min
